# Experiments part

- backbone algo.: PPO / PPO+LSTM

### Q1: The impact of different normalization mechanisms on final performance.

- candidates: vanilla, running mean and std, min-max
- reward: RND (counted-based with representation learning), ICM (Curiosity-driven with representation learning), RE3 (information theory-based without representation learning)
    -- Q: Why only this 3?
- env: 9 atari games, 16 procgen games, mario games
    -- This amount of environments should be more than enough

### Q2: The impact of different integration pattern on final performance. (for on-policy setting)

- vanilla version: ext. reward + int. reward -> adv. estimation -> policy update, only one branch in the value network.
- cleanrl version: separate adv. estimation and two branches in the value network
- RE3 version: multiply the estimated advantages by the intrinsic rewards. -- Q: Multiply or add? 

### Q3: The performance of mixed intrinsic rewards (different/same types).

- candidates: mixture of different/same types of intrinsic rewards (especially episodic+global intrinsic rewards)

### Q4: Is memory required for better optimizing intrinsic rewards?
- LSTM policy vs CNN policy (e.g. E3B doesnt work well without LSTM)

### Q5: Which intrinsic rewards generalize better in Contextual MDPS?
- Compare the performance of all algorithms in fixed envs (e.g. atari game, where env.reset() always takes you to the same worl )
- With the performance in contextual envs (e.g. procgen game, where env.reset() takes you to different worlds, or mario levels (train in a subset, evaluate in hidden test levels))